In [1]:
from backtester import Backtester
import pandas as pd
import numpy as np

### Input parameters for Backtester: asset data frame, strategy function, starting balance (optional)

In [2]:
# Read a sample asset data as data frame
df = pd.read_csv('data_2020_2023_3min.csv')
# Shorten the data frame to 10k rows
df = df.iloc[:1000000]
display('Asset Data Frame', df, df.dtypes)

# Create a sample strategy function that takes asset df as input and outputs list of non-overlapping trade signals
def alternating_trading_strategy(df, trade_interval=10):
    """
    A simple trading strategy that alternates between long and short trades at a fixed interval, ensuring each trade is closed.

    Parameters:
        df (pd.DataFrame): The dataframe containing market data.
        trade_interval (int): The number of periods between trades.

    Returns:
        list: A list of trading signals, where `1` represents a buy signal, `-1` represents a sell signal,
              and `0` represents no action.
    """
    signals = [0] * len(df)  # Initialize all signals to 0
    in_trade = False  # Indicates if currently in a trade
    trade_type = 1  # Start with a long trade

    for i in range(0, len(df)):
        if i % trade_interval == 0:
            signals[i] = trade_type
            in_trade = not in_trade  # Toggle the in_trade status
            if in_trade:
                # If we just entered a trade, prepare to close it in the next interval
                trade_type = -trade_type
        else:
            signals[i] = 0

    # Close any open trade at the end
    if in_trade:
        signals[-1] = -trade_type

    return signals

# Apply the strategy on the asset df and generate list of signals
signals = alternating_trading_strategy(df)
# Display unique signals, positions and their frequencies
signal_series = pd.Series(signals)
position_series = signal_series.cumsum()
signals_df = pd.DataFrame({'Signals': signal_series.value_counts()})
positions_df = pd.DataFrame({'Positions': position_series.value_counts()})
display('Signals', signals_df, 'Positions', positions_df)

'Asset Data Frame'

,datetime,open,high,low,close,volume
0,2020-01-01 05:30:00,7195.24,7196.25,7180.26,7182.43,70.572637
1,2020-01-01 05:33:00,7183.83,7188.94,7178.20,7179.99,37.399739
2,2020-01-01 05:36:00,7180.00,7187.74,7179.99,7187.68,30.886999
3,2020-01-01 05:39:00,7187.68,7193.53,7186.02,7188.71,38.880878
4,2020-01-01 05:42:00,7189.52,7189.52,7180.24,7180.97,25.202615
...,...,...,...,...,...,...
700496,2024-01-01 05:18:00,42267.89,42273.63,42247.87,42249.06,70.344940
700497,2024-01-01 05:21:00,42249.06,42249.07,42221.22,42228.85,36.702670
700498,2024-01-01 05:24:00,42228.86,42240.93,42222.10,42240.93,30.495130
700499,2024-01-01 05:27:00,42240.92,42283.59,42240.92,42283.58,56.903090


datetime     object
open        float64
high        float64
low         float64
close       float64
volume      float64
dtype: object

'Signals'

,Signals
0,630450
-1,35026
1,35025


'Positions'

,Positions
0,350250
1,175130
-1,175121


### See the backtester in action

In [3]:
bt = Backtester(asset_df=df, strategy=alternating_trading_strategy)

# Show trade book
display('Trade Book', bt.tb_df)

# Generate trade ledgers with different modes
bt.generate_ledger(mode='static')
bt.generate_ledger(mode='compounding')

# Show trade ledgers
display('Static Ledger', bt.sl_df)
display('Compounding Ledger', bt.cl_df)

# Summarise backtest results, view long and short trades separately
bt.generate_summary_df(positional_components=True)
display('Backtest Summary', bt.summary_df)

'Trade Book'

,start_time,end_time,position,entry,crest,trough,end,win,change_p,spike_p,dip_p,duration
0,2020-01-01 05:30:00,2020-01-01 06:00:00,1,7182.43,7193.53,7175.47,7183.69,1,0.017543,0.154544,-0.096903,0 days 00:30:00
1,2020-01-01 06:30:00,2020-01-01 07:00:00,-1,7185.45,7180.18,7230.00,7218.65,0,-0.462045,0.073343,-0.620003,0 days 00:30:00
2,2020-01-01 07:30:00,2020-01-01 08:00:00,1,7216.91,7238.88,7211.41,7232.77,1,0.219762,0.304424,-0.076210,0 days 00:30:00
3,2020-01-01 08:30:00,2020-01-01 09:00:00,-1,7230.25,7220.10,7241.40,7241.39,0,-0.154075,0.140382,-0.154213,0 days 00:30:00
4,2020-01-01 09:30:00,2020-01-01 10:00:00,1,7226.76,7229.88,7215.03,7226.31,0,-0.006227,0.043173,-0.162313,0 days 00:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...
35020,2024-01-01 00:30:00,2024-01-01 01:00:00,1,42652.34,42683.99,42586.91,42615.00,0,-0.087545,0.074205,-0.153403,0 days 00:30:00
35021,2024-01-01 01:30:00,2024-01-01 02:00:00,-1,42623.62,42561.10,42680.36,42575.33,1,0.113294,0.146679,-0.133119,0 days 00:30:00
35022,2024-01-01 02:30:00,2024-01-01 03:00:00,1,42591.95,42670.44,42574.49,42600.81,1,0.020802,0.184284,-0.040994,0 days 00:30:00
35023,2024-01-01 03:30:00,2024-01-01 04:00:00,-1,42506.36,42426.83,42591.10,42571.50,0,-0.153248,0.187101,-0.199358,0 days 00:30:00


'Static Ledger'

,position,change_p,deployed,balance,pnl
0,1,0.017543,10000,10001.754281,1.754281
1,-1,-0.462045,10000,9955.549798,-46.204483
2,1,0.219762,10000,9977.525963,21.976164
3,-1,-0.154075,10000,9962.118473,-15.407489
4,1,-0.006227,10000,9961.495788,-0.622686
...,...,...,...,...,...
35020,1,-0.087545,10000,21909.296270,-8.754502
35021,-1,0.113294,10000,21920.625669,11.329399
35022,1,0.020802,10000,21922.705874,2.080205
35023,-1,-0.153248,10000,21907.381109,-15.324766


'Compounding Ledger'

,position,change_p,deployed,balance,pnl
0,1,0.017543,10000.000000,10001.754281,1.754281
1,-1,-0.462045,10001.754281,9955.541693,-46.212588
2,1,0.219762,9955.541693,9977.420155,21.878462
3,-1,-0.154075,9977.420155,9962.047455,-15.372699
4,1,-0.006227,9962.047455,9961.427133,-0.620322
...,...,...,...,...,...
35020,1,-0.087545,19950.651206,19933.185405,-17.465802
35021,-1,0.113294,19933.185405,19955.768506,22.583101
35022,1,0.020802,19955.768506,19959.919715,4.151210
35023,-1,-0.153248,19959.919715,19929.331606,-30.588109


'Backtest Summary'

,n_trades,accuracy_p,aagr_p,cagr_p,static_drawdown_p,comp_drawdown_p,max_spike_p,max_dip_p,max_win_p,avg_win_p,max_loss_p,avg_loss_p,avg_rr,worst_rr,max_duration,average_duration,min_duration
long_trades,17513,50.665220,0.001904,23.608640,49.561276,77.062070,16.422460,-18.157023,15.807886,0.310033,-14.436982,-0.302964,0.977199,46.565888,0 days 04:00:00,0 days 00:30:01.418374921,0 days 00:30:00
short_trades,17512,49.965738,-0.000206,-3.821762,49.770702,76.835964,13.966497,-16.803410,7.908434,0.312872,-11.006655,-0.314090,1.003892,35.179418,0 days 02:30:00,0 days 00:30:00.719506624,0 days 00:30:00
all_trades,35025,50.315489,0.000849,21.650966,49.835716,77.062070,16.422460,-18.157023,15.807886,0.311443,-14.436982,-0.308566,0.990763,46.355162,0 days 04:00:00,0 days 00:30:01.068950749,0 days 00:30:00
